In [33]:
%matplotlib qt5
#%% Import modules
from mats_utils.rawdata.read_data import read_MATS_data
import datetime as DT
from mats_utils.plotting.plotCCD import *
from math import *
from mats_l1_processing.pointing import pix_deg
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.spatial.transform import Rotation as R
from scipy.optimize import minimize_scalar
from skyfield.positionlib import ICRF
from skyfield.api import wgs84 
from skyfield.units import Distance
from skyfield import api as sfapi
from scipy.interpolate import RegularGridInterpolator

In [38]:
def pix_deg(ccditem, xpixel, ypixel):
    """ 
    Inputs : CCDitem  , xpixel (can be an array), ypixel (can be an array)
    Outputs xdeg - x offset of the pixel(s) in degrees, ydeg - y offset of the pixels(s) in degrees

    """
    xdisp = 6.06/2047
    ydisp = 1.52/510  # 1.52/510
    ncskip = ccditem['NCSKIP']
    try:
        ncbin = ccditem['NCBIN CCDColumns']
    except:
        ncbin = ccditem['NCBINCCDColumns']
    nrskip = ccditem['NRSKIP']
    nrbin = ccditem['NRBIN']
    ncol = ccditem['NCOL']
    # flipped configuration: clips from right before flip left clip by limiting no. of  columns
    if (ccditem['CCDSEL']) in [1, 3, 5, 6]:
        xdeg = xdisp*((2048-ncskip - (ncol+1)*ncbin +
                      ncbin*(xpixel+0.5)) - 2047./2)
    else:
        xdeg = xdisp*(ncskip + ncbin * (xpixel+0.5) - 2047./2)
    ydeg = ydisp*(nrskip + nrbin * (ypixel+0.5) - 510./2)

    tmpx = xdeg/xdisp
    tmpy = ydeg/ydisp

    return xdeg, ydeg

def pix_deg2(ccditem, xpixel, ypixel):
    """
    Function to get the x and y angle from a pixel relative to the center of the CCD
    WARNING : no images are flipped in this function
    
    Parameters
    ----------
    ccditem : CCDitem
        measurement
    xpixel : int or array[int]
        x coordinate of the pixel(s) in the image
    ypixel : int or array[int]
        y coordinate of the pixel(s) in the image
        
    Returns
    -------
    xdeg : float or array[float]
        angular deviation along the x axis in degrees (relative to the center of the CCD)
    ydeg : float or array[float]
        angular deviation along the y axis in degrees (relative to the center of the CCD) 
    """
    h = 6.9 # height of the CCD in mm
    d = 27.6 # width of the CCD in mm
    # selecting effective focal length
    if (ccditem['CCDSEL']) == 7: # NADIR channel
        f = 50.6 # effective focal length in mm
    else: # LIMB channels
        f = 261    
    
    ncskip = ccditem['NCSKIP']
    try:
        ncbin = ccditem['NCBIN CCDColumns']
    except:
        ncbin = ccditem['NCBINCCDColumns']
    nrskip = ccditem['NRSKIP']
    nrbin = ccditem['NRBIN']
    ncol = ccditem['NCOL'] # number of columns in the image MINUS 1

    y_disp = (h/(f*511))
    x_disp = (d/(f*2048))
  
    if (ccditem['CCDSEL']) in [1, 3, 5, 6]:
        xdeg = np.rad2deg(np.arctan(x_disp*((2048-ncskip - (ncol+1)*ncbin + ncbin*(xpixel+0.5)) - 2047./2)))
    else:
        xdeg = np.rad2deg(np.arctan(x_disp*(ncskip + ncbin * (xpixel+0.5) - 2047./2)))
        
    ydeg = np.rad2deg(np.arctan(y_disp*(nrskip + nrbin * (ypixel+0.5) - 510./2)))

    return xdeg, ydeg


def deg_map(ccditem):
    """
    Function to get the x and y angular deviation map for each pixel of the image. 
    The deviation is given in degrees relative to the center of the CCD
    WARNING : no images are flipped before calculating the angular deviation
    
    Parameters
    ----------
    ccditem : CCDitem
        measurement
            
    Returns
    -------
    xmap : array[float]
        angular deviation map along the x axis in degrees (relative to the center of the CCD)
    ymap : array[float]
        angular deviation map along the y axis in degrees (relative to the center of the CCD) 
    """    
    im = ccditem['IMAGE']

    a,b = np.shape(im)
    X = range(b)
    Y = range(a)
    xpixel, ypixel = np.meshgrid(X,Y)
    xmap,ymap = pix_deg2(ccditem, xpixel, ypixel)
    return xmap,ymap

def deg_map_test(ccditem):
    """
    Function to get the x and y angular deviation map for each pixel of the image. 
    The deviation is given in degrees relative to the center of the CCD
    WARNING : no images are flipped before calculating the angular deviation
    
    Parameters
    ----------
    ccditem : CCDitem
        measurement
            
    Returns
    -------
    xmap : array[float]
        angular deviation map along the x axis in degrees (relative to the center of the CCD)
    ymap : array[float]
        angular deviation map along the y axis in degrees (relative to the center of the CCD) 
    """    
    im = ccditem['IMAGE']

    a,b = np.shape(im)
    X = range(b)
    Y = range(a)
    xpixel, ypixel = np.meshgrid(X,Y)
    xmap,ymap = pix_deg(ccditem, xpixel, ypixel)
    return xmap,ymap


In [51]:
#%% local variables
# value of a saturated pixel 
sat_val = 32880

# times for start and stop
start_time = DT.datetime(2023, 1, 12, 3, 30, 0)
stop_time = DT.datetime(2023, 1, 12, 4, 0, 0)
start_time = DT.datetime(2023, 3, 13, 0, 0, 0)
stop_time = DT.datetime(2023, 3, 13, 3, 30, 30)

# filter selecting Nadir chanel
filter={'CCDSEL': [6,6]}
#%% reading mesurements
df1a= read_MATS_data(start_time, stop_time,filter,level='1a',version='0.5')
print(len(df1a))

# displaying keys
pd.set_option('display.max_rows', 100)
df1a.dtypes

Warning: Dataset is empty check version or time interval

In [ ]:
ccditem = df1a.iloc[0]
x,y = deg_map(ccditem)
xt,yt = deg_map_test(ccditem)
dx = x-xt
dy = y-yt
print(np.min(dx))
print(np.max(dx))
print(np.min(dy))
print(np.max(dy))

-0.002885522586998146
0.003958846958612927
-0.0041512647642900635
0.001912754269756456
